Retomando los datos con los que hemos estado trabajando hasta ahora ...

In [ ]:
! pip install -U spacy
! python -m spacy download es_core_news_lg

In [1]:
import spacy
nlp = spacy.load("es_core_news_lg")

Para las secciones `1`, `2` y `3` utilizaremos funciones creadas en sesiones pasadas.

## 1. Carga de datos


In [2]:
def cargar_datos(ruta_con_datos):
  import pandas as pd
  datos = pd.read_csv(ruta_con_datos, sep="\t")
  return datos

In [3]:
#ruta = 'datos_deteccion_sexismo'
ruta = '/datos_deteccion_sexismo.tsv'
datos = cargar_datos(ruta)

In [4]:
datos

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality
1,EXIST2021,2,twitter,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist
2,EXIST2021,3,twitter,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification
3,EXIST2021,4,twitter,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist
...,...,...,...,...,...,...,...
6972,EXIST2021,6973,twitter,es,"Estamos igual sin pareja, pero puedes besar a ...",non-sexist,non-sexist
6973,EXIST2021,6974,twitter,es,2020 hijo de re mil putas,non-sexist,non-sexist
6974,EXIST2021,6975,twitter,es,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,non-sexist,non-sexist
6975,EXIST2021,6976,twitter,es,@safetyaitana mi madre dice q va fea y i agree,sexist,objectification


# 2. Eliminación de columnas

In [5]:
def elimina_columna_en_pandas(variable_pandas, columnas_a_eliminar):
  import pandas as pd
  variable_pandas.drop(columnas_a_eliminar, axis=1, inplace=True)
  return variable_pandas

In [6]:
columnas_a_eliminar = ['test_case', 'id', 'source']
datos_nuevos = elimina_columna_en_pandas(datos, columnas_a_eliminar)

In [7]:
# Revisamos que efectivamente las columnas test_case, id y source hayan sido
# eliminadas de la variable datos_nuevos.
datos_nuevos

,language,text,task1,task2
0,en,"She calls herself ""anti-feminazi"" how about sh...",sexist,ideological-inequality
1,en,"Now, back to these women, the brave and the be...",non-sexist,non-sexist
2,en,"@CurvyBandida @Xalynne_B Wow, your skirt is ve...",sexist,objectification
3,en,@AurelieGuiboud Incredible! Beautiful!But I l...,non-sexist,non-sexist
4,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist
...,...,...,...,...
6972,es,"Estamos igual sin pareja, pero puedes besar a ...",non-sexist,non-sexist
6973,es,2020 hijo de re mil putas,non-sexist,non-sexist
6974,es,SEGURAMENTE ESTA CHICA NO COBRA EL DINERO QUE ...,non-sexist,non-sexist
6975,es,@safetyaitana mi madre dice q va fea y i agree,sexist,objectification


## 3. Eliminación de tuits en inglés

In [8]:
datos_espanol = datos_nuevos[datos_nuevos['language'] != 'en']

In [9]:
# Revisamos el tuit en el índice 3532 en la columna text de la variable datos_espanol.
datos_espanol['text'][3532]

'Miranda Cosgrove paso de ser una perra mala a una mojigata aljsdalkjdsd https://t.co/uT8Sb7dr2g'

In [10]:
# Revisamos qué tokens del tuit en el índice 3532 en la columna text de la variable datos_espanol
# no reconoce spaCy. Recordemos que spaCy marca un token como token.is_oov == True si ese token
# no se encuentra dentro de los tokens que se le metieron al modelo durante su entrenamiento.
doc = nlp(datos_espanol['text'][3532])
for token in doc:
  if token.is_oov == True:
    print(token.text)

aljsdalkjdsd
https://t.co/uT8Sb7dr2


Observamos que hay dos tokens que spaCy marca como fuera de vocabulario (out of vocabulary): `aljsdalkjdsd` y `https://t.co/uT8Sb7dr2`. Esto era de esperarse, pues difícilmente un conjunto de datos de notas periodísticas (que es el qe se indica en la página de spaCy que se usó para entrenar el modelo que estamos usando en esta notebook) contendría estos dos términos,

## 4. Funciones para corrección de ortografía

La función `corrige_errores_ortograficos` ayuda a la función `aplica_diccionario` a introducir sustituciones ortográficas manualmente a un conjunto de datos.

En este caso se usa `aplica_diccionario` con la variable `datos_espanol` sobre la columna `text`. También se le pasa como argumento `diccionario_corregido`, el cual contiene pares de palabras: la primera palabra es la que queremos sustituir, y la que esta después de los `:` es por la que la queremos sustituir.

Para modificar estas reglas de sustitución ortográfica, simplemente debemos agregar manualmente los pares *sustituir-sustitución* dentro de la variable `diccionario_corregido`. Se pueden ingresar tantos pares se desee.


In [16]:
def corrige_errores_ortograficos(x, dic):
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def aplica_diccionario(datos, columna, diccionario):
  datos[columna] = datos[columna].apply(lambda x: corrige_errores_ortograficos(x, diccionario))
  return 

In [ ]:
diccionario_corregido = {'aljsdalkjdsd': 'jajaja', 'https://t.co/uT8Sb7dr2g': 'link'}

aplica_diccionario(datos_espanol, 'text', diccionario_corregido)

Verificamos que efectivamente se hayan sustituido los términos *aljsdalkjdsd* y *https://t.co/uT8Sb7dr2g* por *jajaja* y *link*:

In [18]:
datos_espanol['text'][3532]

'Miranda Cosgrove paso de ser una perra mala a una mojigata jajaja link'

## 5. Etiquetar las tokens en los tuits

En esta sección definieremos clases de tokens manualmente. Hasta ahora sólo hemos extraído las tokens que spaCy tiene por defecto. Si queremos introducirle al modelo algún tipo de token en particular debemos hacer lo siguiente:

In [19]:
def crea_tipo_de_token(nombre_tipo_token, ejemplos):
  from spacy.tokens import Token
  tokens_nuevos = lambda token: token.text in ("Rosalía", "Madonna")
  Token.set_extension(str(nombre_tipo_token), getter=tokens_nuevos, force=True)
  return 

En la celda anterior obtenemos un error, pues el modelo de spaCy que cargamos en esta libreta NO contiene el token `es_musico`. 

Por lo tanto introducimos este tipo de token con la celda siguiente:

In [20]:
es_musico = 'es_musico'
musicos = ("Rosalía", "BTS")

In [21]:
doc = nlp('Me gusta Rosalía.')

## Llamamos a la función crea_tipo_token y le pasamos nuestros parámetros definidos
# en la celda anterior.
crea_tipo_de_token(es_musico, musicos)

Finalmente revisamos que el token `Rosalía` efectivamente haya sido etiquetado como `es_musico`; esto es, que `token._.es_musico` devuelva `True` sólo para el `token.text == Rosalía`.

In [22]:
print('Tokens | Si es True significa que el token es de tipo .es_musico')
for token in doc:
  print(token.text, token._.es_musico)

Tokens | Si es True significa que el token es de tipo .es_musico
Me False
gusta False
Rosalía True
. False


Ahora introduciremos otra regla manualmente al modelo de spaCy cargado en esta libreta. 

Esta vez le diremos que al generar los `doc.sents` (recordemos: estos son tokens por oración y no por palabra como los que genera `token.text`) segmente las oraciones cuando encuentre un `...`

In [23]:
from spacy.language import Language
import spacy

texto = "esta es una oración...ahora...ésta es otra."

doc = nlp(texto)
print("Antes:", [sent.text for sent in doc.sents])

Antes: ['esta es una oración...ahora...ésta es otra.']


En la celda enterior observamos que `doc.sents` genera sólo una lista con un token de oración, pues no considera que `...` separe a dos oraciones.

En el siguiente bloque de código definimos una función que luego introducimos a spaCy con la ayuda de `nlp.add_pipe`. Para cambiar el funcionamiento de este bloque de código e introducir otra regla a partir de la cual segmentar oraciones (diferente a `...`), sólo hay que cambiar la línea `4` por nuestra nueva regla.

In [24]:
@Language.component("limites_manuales_de_oraciones")
def limites_manuales_de_oraciones(doc):
    for token in doc[:-1]:
        if token.text == "...":
            doc[token.i + 1].is_sent_start = True
    return doc

nlp.add_pipe("limites_manuales_de_oraciones", before="parser")

doc = nlp(texto)
print("Después:", [sent.text for sent in doc.sents])

Después: ['esta es una oración...', 'ahora...', 'ésta es otra.']


Lo siguiente que haremos en esta libreta es revisar cuántos tokens fuera de vocabulario (out of vocabulary - oov) tenemos entre nuestros datos en la variable `datos_espanol`, columna `text`.

In [25]:
palabras_fuera_de_vocabulario = []

for tuit in datos_espanol['text']:
  doc = nlp(tuit)
  for token in doc:
    if token.is_oov == True:
      palabras_fuera_de_vocabulario.append(token.text)

In [26]:
palabras_fuera_de_vocabulario

['@lindagisela74',
 '@BicireporteraDF',
 '@Agus_Martinez58',
 '@AlmeidaPP',
 'ALCALDESPRETENDEN',
 'CABELO',
 'CABELA',
 'ESTRECHA',
 'EXPRESASUBTERRANEA',
 'https://t.co/xY9smufSdS',
 'https://t.co/rhuvSXhypu',
 '@6Asca',
 '@ladycristall',
 'lagartona',
 'conchalalora',
 'https://t.co/N19KHrYztB',
 '@houstontexans',
 '#nfl',
 'futbolamericano',
 'willfuller',
 'texans',
 'texansnation',
 'pedpolicy',
 'https://t.co/cD3y8SyslO',
 'bailando.son',
 'NiUnEuroACruzRoja',
 'preciones',
 '@Rexmas1alterno',
 '@AdryMJ',
 ' ',
 'https://t.co/Cb4Ldqlxgv',
 'sexualicen',
 '@marianavelezga',
 'guebon',
 '@AKO72842870',
 ' ',
 '@Noticia_Libre',
 '@JulySeEs',
 '@AnibelcaRosario',
 '@SusanaGautreau',
 '@holi_matos',
 '@pousuazo',
 '@laluzjose',
 '@carlosgabrielgc',
 '@LaNacion_RD',
 '@SueroDanielUnt',
 '@Porfiriogc',
 '@Suedita',
 '@LaVozDelPRM',
 'nadie?Que',
 'SALISTE',
 'PARECES',
 'https://t.co/dS5baUS4Jw',
 '@EdwardMa2558',
 '@Cydonia99790009',
 '@La_SER',
 '@adelamolina',
 '@elmundoes',
 '@Goog

Tiene sentido que los tokens deplegados en la celda anterior no estén dentro del modelo de spaCy, pues son arrobas de twitter, y links (en su mayoría).


Ahora obtengamos el porcentaje de token fuera de vocabulario. Para esto, primero meteremos en la lista `palabras` todos los tokens en `datos_espanol['text']`. 

Luego usaremos la longitud de la lista `palabras`, y la de la lista `palabras_fuera_de_vocabulario` para obtener el pocentaje de tokens fuera de vocabulario.


In [ ]:
palabras = []
for tuit in datos_espanol['text']:
  for palabra in tuit:
    palabras.append(palabras)

In [ ]:
print(len(palabras), len(palabras_fuera_de_vocabulario))

623536 8191


In [ ]:
((8191*100)/623536)

1.3136370634574428

Observamos que sólo tenemos `1.3%` tokens fuera de vocabulario. 

Al ser un porcentaje muy bajo, dejaremos esos tokens en la variable `datos_espanol`.


Después del análisis anterior crearemos dos listas; una con los tuits originales, y otra con los tuits que contienen tokens marcados como fuera de vocabulario. 

La lista con los tokens fuera de vocabulario los sustituiremos por el texto `TOKEN ELIMINADA`. Estos cambios NO afectarán a la variable `datos_espanol`, pues los haremos en la nueva lista creada (`tuits_viejos`).

In [ ]:
tuits_nuevos = []
tuits_viejos = []

for tuit, palabra in zip(datos_espanol['text'], palabras_fuera_de_vocabulario):
  if palabra in tuit:
    tuits_nuevos.append(tuit.replace(palabra, 'TOKEN ELIMINADA'))
    tuits_viejos.append(tuit)

In [ ]:
len(tuits_nuevos)

185

In [ ]:
tuits_nuevos

['LA ALCALDESA CLAUDIA LOPEZ Y DEMAS ALCALDESPRETENDEN METER POR LA SEPTIMA LO QUE NO CABELO QUE NO CABE NO TOKEN ELIMINADA SEPTIMA DESDE LA CALLE 32 A LA CALLE 100 ES MUY ESTRECHA HAY QUE HACER UNA VIA EXPRESASUBTERRANEA O AEREA https://t.co/xY9smufSdS https://t.co/rhuvSXhypu',
 'MiTOKEN ELIMINADAcuerpoTOKEN ELIMINADAnoTOKEN ELIMINADAseTOKEN ELIMINADAtoca,TOKEN ELIMINADAnoTOKEN ELIMINADAseTOKEN ELIMINADAviola,noTOKEN ELIMINADAseTOKEN ELIMINADAmata.SiTOKEN ELIMINADAconocesTOKEN ELIMINADAalgúnTOKEN ELIMINADAcasoTOKEN ELIMINADAoTOKEN ELIMINADAsufresTOKEN ELIMINADAdeTOKEN ELIMINADAviolenciaTOKEN ELIMINADADENUNCIA.EstamosTOKEN ELIMINADAparaTOKEN ELIMINADAayudarteTOKEN ELIMINADA#PorTíPorMíPorTodas#DenunciaLaViolenciaTOKEN ELIMINADA#JusticiaTOKEN ELIMINADA#YoSiTeCreoTOKEN ELIMINADA#NoEsNoTOKEN ELIMINADAhttps://t.co/eDVDG9tWtG',
 '¡ATOKEN ELIMINADAti,TOKEN ELIMINADAmujerTOKEN ELIMINADAluchadora!TOKEN ELIMINADA(JorgeTOKEN ELIMINADAAcuña)"MujerTOKEN ELIMINADAtrabajadora,TOKEN ELIMINADArebeldeTO

Los tuits mostrados en la celda anterior son los tuits editados dentro de la lista `tuits_nuevos`. Como dicha lista sólo tiene una longitud de `185`, no quitaremos dichos tuits de la variable `datos_espanol`.

# `huggingface`

La librería [hugginface](https://huggingface.co/) permite cargar modelos pre-entrenados por diversas empresas y personas. En su página especifican el nombre de cada modelo, y qué datos se usaron para entrenar dichos modelos. También suelen mostrar ejemplos del funcionamiento de los modelos.

Recordemos que spaCy no es la única empresa que genera modelos. 

In [ ]:
# Como siempre, primero debemos descargar la librería a la libreta.
! pip install transformers

In [ ]:
# Importamos la librería recientemente descargada. En este caso importamos la función
# pipeline, pues esta nos permite especificar tareas de procesamiento del lenguaje natural.
from transformers import pipeline

# Carguemos la tarea análisis de sentimientos en la variable classifier.
classifier = pipeline("sentiment-analysis") 

# A la variable classifier, que tiene ya cargado un modelo (no especificado
# manualmente en este caso), le pasamos un par de oraciones para ver qué
# etiquetas les asigna. El modelo distilbert-base-uncased-finetuned-sst-2-english,
# que viene precargado por defecto, devuelve dos tipo de etiquetas: POSITIVE y 
# NEGATIVE. Recordemos que score se refiere a probabilidades que los modelos
# asignan a cada etiqueta. Por ejemplo, la primera oración, etiquetada como
# POSITIVE, tiene una probabilidad de 95.9% de ser correcta. 
# Recorda: Estos modelos son estadísiticos siempre. No hay ningún modelo de
# inteligencia artificial que sea capaz de asegurar algo sin margen de error.
classifier(
    ["I've been waiting for a HuggingFace course my whole life.",
        "I hate this so much!",])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [ ]:
from transformers import pipeline, set_seed

# Ahora cargaremos el modelo nlptown/bert-base-multilingual-uncased-sentiment a
# la variable clasificador. Esto porque dicho modelo parece ser más adecuado 
# a los tuits en español que tenemos en esta libreta.
clasificador = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

# Probemos este modelo en la tarea de análisis de sentimientos con el texto "Te quiero".
texto = 'Te quiero.'
clasificador([texto])

[{'label': '5 stars', 'score': 0.512648344039917}]

In [ ]:
from transformers import pipeline, set_seed

# Probemos al modelo gpt2 en la tarea de generación de texto. Recordemos que 
# hay modelos específicos para generación de texto, y otros para realizar clasificaciones.
# Si aquí pasáramos al modelo nlptown/bert-base-multilingual-uncased-sentiment 
# habría un error pues ese modelo es de clasificación, no de generación.
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("The White man worked as a", max_length=10, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The White man worked as a clerk at the old'},
 {'generated_text': 'The White man worked as a salesman in Mexico and'},
 {'generated_text': 'The White man worked as a lawyer in the White'},
 {'generated_text': 'The White man worked as a clerk for the store'},
 {'generated_text': 'The White man worked as a barkeep and was'}]

In [ ]:
set_seed(42)
generator("The Black man worked as a", max_length=10, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The Black man worked as a clerk at the old'},
 {'generated_text': 'The Black man worked as a salesman in Mexico and'},
 {'generated_text': 'The Black man worked as a lawyer in the city'},
 {'generated_text': 'The Black man worked as a clerk for the store'},
 {'generated_text': 'The Black man worked as a barkeep and was'}]

In [ ]:
# Repitamos lo anterior pero con un modelo gpt2 en español.
generator = pipeline('text-generation', model='DeepESP/gpt2-spanish')
set_seed(42)
generator("El hombre blanco trabajó como", max_length=10, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'El hombre blanco trabajó como el que más, y'},
 {'generated_text': 'El hombre blanco trabajó como si fuera una máquina y'},
 {'generated_text': 'El hombre blanco trabajó como barquero en uno de'},
 {'generated_text': 'El hombre blanco trabajó como un dios, desde el'},
 {'generated_text': 'El hombre blanco trabajó como un autómata; no'}]

In [ ]:
set_seed(42)
generator("El hombre negro trabajó como", max_length=10, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'El hombre negro trabajó como el que más, y'},
 {'generated_text': 'El hombre negro trabajó como si fuera una máquina y'},
 {'generated_text': 'El hombre negro trabajó como barquero en uno de'},
 {'generated_text': 'El hombre negro trabajó como un dios, desde el'},
 {'generated_text': 'El hombre negro trabajó como un autómata; no'}]

## Volviendo al corpus de detección de sexismo

En la siguiente sección realizaremos los siguientes dos pasos:

1. Extraer los tuits en cada línea de la variable `datos_espanol` columna `text`. Los guardaremos en la lista `texto`.
2. Cargar la tarea `sentiment-analysis` con el modelo `nlptown/bert-base-multilingual-uncased-sentiment` que se encuentra en huggingface. Esto se guarda en la variable `clasificador` usando la función `pipeline` de huggingface.
3. Pasamos por `clasificador` la lista creada en el paso 1. Esto hará que el modelo cargado en 2. asigne una etiqueta de negatividad/positivadd a cada tuit. Esas etiquetas las guardamos en la lista `evaluaciones` para poder consultarlas después.

In [ ]:
texto = []
for linea in datos_espanol['text']:
  texto.append(linea)

In [ ]:
# Revisamos el contenido de la variable texto para confirmar que sí tiene los
# tuits de la variable datos_espanol columna text.
texto

In [ ]:
clasificador = pipeline('sentiment-analysis', model='nlptown/bert-base-multilingual-uncased-sentiment')

evaluaciones = []
evaluaciones.append(clasificador(texto))

### Lecturas sugeridas (en inglés)

1. [Look behind the curtain: Don’t be dazzled by claims of ‘artificial intelligence'](https://www.seattletimes.com/opinion/look-behind-the-curtain-dont-be-dazzled-by-claims-of-artificial-intelligence/)

2. [Timnit Gebru was fired from Google - Then the harrasers arrived](https://www.theverge.com/22309962/timnit-gebru-google-harassment-campaign-jeff-dean) 

